In [6]:
import numpy as np
import copy
from nltk.tokenize import sent_tokenize
import torch
import torch.optim as optim
from torch import nn



In [7]:
data = open('bai_van_mau.txt','r',encoding='utf-8').read()
data = data.replace('\n', ' ').lower()
chars = list(set(data))
sents = sent_tokenize(data)
sents = [sent.replace('.','') for sent in sents]

char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
datasize, sents_size, vocab_size =  len(data), len(sents), len(chars)
datasize, sents_size, vocab_size
max_str_len = len(max(sents,key=len))
hidden_size = 100

In [4]:
def initialize_parameters(n_a, n_x, n_y):
    """
    Initialize parameters with small random values
    
    Returns:
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    """
    Wax = torch.randn(n_a, n_x)*0.01 # input to hidden
    Waa = torch.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = torch.randn(n_y, n_a)*0.01 # hidden to output
    b = torch.zeros((n_a, 1), requires_grad=True) # hidden bias
    by = torch.zeros((n_y, 1), requires_grad=True) # output bias
    
    Wax.requires_grad=True
    Waa.requires_grad=True
    Wya.requires_grad=True
    
    parameters = [Wax, Waa, Wya, b, by]
    
    return parameters

# test init paramter
parameters = initialize_parameters(50, vocab_size, vocab_size)

In [5]:
def softmax1d(x):
    max, _ = torch.max(x, dim=0, keepdims=True) #returns max of each row and keeps same dims
    e_x = torch.exp(x - max) #subtracts each row with its max value
    sum = torch.sum(e_x, dim=0, keepdims=True) #returns sum of each row and keeps same dims
    sf = e_x / sum 

    return sf
# test
a = torch.randn(4,1)
softmax1d(a)

tensor([[0.4397],
        [0.0908],
        [0.0950],
        [0.3745]])

In [6]:
def get_sample(sample_ix):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    return txt

In [7]:
def rnn_step_forward(parameters, a_prev, x):
    Wax, Waa, Wya, b, by = parameters
    a = torch.tanh(Wax @ x + Waa@ a_prev + b)
    z = Wya @ a + by
#     y = softmax1d(z)
    
    return a, z

# check
a_prev = torch.zeros((n_a,1))
x = torch.zeros((vocab_size, 1))
a, y = rnn_step_forward(parameters, a_prev, x)
a.shape,y.shape

(torch.Size([50, 1]), torch.Size([136, 1]))

In [20]:
def sample(parameters, maxlength=200):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN
   """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters
    n_y, n_a = Wya.shape

    x = torch.zeros((vocab_size, 1))
    x[np.random.randint(0,vocab_size)] = 1
    a_prev = torch.zeros((n_a,1))
    
    indices = []
    idx = -1 
    
    counter = 0
    newline_character = char_to_ix['.']
    
    while (idx != newline_character and counter != maxlength):
        a_prev, y = rnn_step_forward(parameters, a_prev, x)
        y = softmax1d(y)
        y_local = y.detach().numpy()
        
#         idx = np.random.choice(range(len(y_local)), p=y_local.reshape(-1))
        idx = np.argmax(y_local)
        indices.append(idx)
        
        # one hot vector
        x = torch.zeros((vocab_size, 1))
        x[idx] = 1
       
        counter +=1

    if (counter == maxlength):
        indices.append(char_to_ix['.'])
    
    return indices

# test sample
n_a, n_x, n_y = 50, vocab_size, vocab_size
parameters = initialize_parameters(n_a, n_x, n_y)
sample(parameters), len(sample(parameters)), get_sample(sample(parameters))


([69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87,
  36,
  13,
  69,
  50,
  114,
  47,
  98,
  1,
  93,
  27,
  10,
  29,
  87

In [32]:
torch.autograd.set_detect_anomaly(False)

In [22]:
def rnn_forward(X, Y, a0, parameters):
    # X n_x, m, t_x
    # t_x = max_str_len
    # Initialize x, a and y_hat as empty dictionaries
    sequence_len = len(X)
    x = torch.zeros((sequence_len, vocab_size, 1))
    y_hat = torch.zeros((sequence_len, vocab_size, 1))
    y = torch.zeros((sequence_len, vocab_size, 1))
    # initialize your loss to 0
    a = torch.zeros((sequence_len+1, n_a, 1))
    a[0] = a0.detach()
    loss = torch.zeros(1)
    for t in range(sequence_len):
        # Set x[t] to be the one-hot vector representation of the t'th character in X.
        # if X[t] == None, we just have x[t]=0. This is used to set the input for the first timestep to the zero vector. 
        
        if (X[t] != None):
            x[t][X[t],0] = 1
        y[t][Y[t]] = 1

    for t in range(0,sequence_len):
        # Run one step forward of the RNN
        a[t+1], y_hat[t] = rnn_step_forward(parameters, a[t].clone(), x[t])
        
    # Update the loss by substracting the cross-entropy term of this time-step from it.
    # loss = torch.sum(-y*torch.log(y_hat))
        
    return y_hat, a[-1]

# test rnn_forward
single_example = sents[0]
single_example_chars = [c for c in single_example]
single_example_ix = [char_to_ix[c] for c in single_example_chars]

X = [None]+single_example_ix
ix_newline = char_to_ix['.']
Y = X[1:] + [ix_newline]
a_prev = torch.zeros((n_a, 1))
y_hat, a = rnn_forward(X, Y, a_prev, parameters)
y_hat.shape, a.shape

(torch.Size([297, 136, 1]), torch.Size([50, 1]))

In [18]:
def optimize(X, Y, a_prev, parameters, optimizer, criterion):
    """
    Execute one step of the optimization to train the model.
    """
    # Forward propagate through time (≈1 line)

    y_hat, a_prev =  rnn_forward(X, Y, a_prev, parameters)
    loss = criterion(y_hat.reshape(-1,vocab_size), torch.LongTensor(Y))
    loss.backward()
    
#     Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    torch.nn.utils.clip_grad_value_(parameters, 5)
    
#     Update parameters (≈1 line)
    optimizer.step()    # Does the update
    optimizer.zero_grad()
    
    return loss, a_prev

# test optimize 
parameters = initialize_parameters(n_a, vocab_size, vocab_size)

single_example = sents[0]
single_example_chars = [c for c in single_example]
single_example_ix = [char_to_ix[c] for c in single_example_chars]

X = [None]+single_example_ix
ix_newline = char_to_ix['.']
Y = X[1:] + [ix_newline]
optimizer = optim.Adam(parameters, lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion = nn.CrossEntropyLoss()
a_prev = torch.zeros((n_a, 1))
for i in range(2):
    loss, a_prev = optimize(X, Y, a_prev, parameters, optimizer, criterion)
a_prev.shape

torch.Size([50, 1])

In [19]:

def model(examples, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, num_sampling = 7, verbose = False):
    """
    Trains the model and generates text. 
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    optimizer = optim.Adam(parameters, lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for item in parameters:
        print(item.shape)
    # Initialize loss (this is required because we want to smooth our loss)
#     loss = get_initial_loss(vocab_size, num_sampling)
        
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize the hidden state 
    a_prev = torch.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        ### START CODE HERE ###
        
        # Set the index `idx` (see instructions above)
        idx = j % len(examples)
        
        # Set the input X (see instructions above)
        single_example = examples[idx]
        single_example_chars = [c for c in single_example]
        single_example_ix = [char_to_ix[c] for c in single_example_chars]
        X = [None]+single_example_ix
        
        # Set the labels Y (see instructions above)
        ix_newline = char_to_ix['.']
        Y = X[1:] + [ix_newline]
        if verbose == True and j < 10:
            print(single_example, " X = ", X, "\n", "Y =       ", Y, "\n")
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        loss, a_prev = optimize(X, Y, a_prev, parameters, optimizer, criterion)

        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
#         loss = smooth(loss, curr_loss.detach())
        
        
        if j % len(examples) == 0:
            np.random.shuffle(examples)
            
        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 200 == 1:
            print('\nIteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            for name in range(num_sampling):
                # Sample indices and print them
                sampled_indices = sample(parameters)
                last_sentence = get_sample(sampled_indices)
                print(last_sentence)

    return parameters

parameters = model(sents, ix_to_char, char_to_ix, 35000, verbose = True)

torch.Size([50, 136])
torch.Size([50, 50])
torch.Size([136, 50])
torch.Size([50, 1])
torch.Size([136, 1])
bài làm  sau kì kiểm tra giữa học kì hai lớp ba, lớp em được nhà trường tổ chức đưa đi tham quan gian viện bảo tàng thành phố  X =  [None, 55, 123, 87, 40, 135, 123, 53, 40, 40, 36, 33, 127, 40, 22, 72, 40, 22, 87, 97, 53, 40, 12, 92, 33, 40, 39, 87, 32, 33, 40, 104, 59, 64, 40, 22, 72, 40, 104, 33, 87, 40, 135, 91, 128, 40, 55, 33, 67, 40, 135, 91, 128, 40, 47, 53, 40, 83, 111, 103, 64, 40, 96, 104, 123, 40, 12, 92, 111, 118, 96, 39, 40, 12, 14, 40, 64, 104, 77, 64, 40, 83, 111, 33, 40, 83, 87, 40, 12, 104, 33, 53, 40, 88, 127, 33, 96, 40, 39, 87, 33, 96, 40, 20, 87, 26, 96, 40, 55, 70, 98, 40, 12, 123, 96, 39, 40, 12, 104, 123, 96, 104, 40, 128, 104, 44] 
 Y =        [55, 123, 87, 40, 135, 123, 53, 40, 40, 36, 33, 127, 40, 22, 72, 40, 22, 87, 97, 53, 40, 12, 92, 33, 40, 39, 87, 32, 33, 40, 104, 59, 64, 40, 22, 72, 40, 104, 33, 87, 40, 135, 91, 128, 40, 55, 33, 67, 40, 135, 91, 12

mỗi ngày, chồng ra biển đánh cá thì vợ ởnhà trông con và vá lưới, đan lưới  X =  [None, 53, 86, 87, 40, 96, 39, 123, 1, 67, 40, 64, 104, 99, 96, 39, 40, 92, 33, 40, 55, 87, 97, 96, 40, 83, 38, 96, 104, 40, 64, 38, 40, 12, 104, 72, 40, 20, 103, 40, 110, 96, 104, 123, 40, 12, 92, 68, 96, 39, 40, 64, 98, 96, 40, 20, 123, 40, 20, 38, 40, 135, 111, 91, 87, 67, 40, 83, 33, 96, 40, 135, 111, 91, 87] 
 Y =        [53, 86, 87, 40, 96, 39, 123, 1, 67, 40, 64, 104, 99, 96, 39, 40, 92, 33, 40, 55, 87, 97, 96, 40, 83, 38, 96, 104, 40, 64, 38, 40, 12, 104, 72, 40, 20, 103, 40, 110, 96, 104, 123, 40, 12, 92, 68, 96, 39, 40, 64, 98, 96, 40, 20, 123, 40, 20, 38, 40, 135, 111, 91, 87, 67, 40, 83, 33, 96, 40, 135, 111, 91, 87, 107] 

ve áo gi-lê đính một ngôi sao bạc, gấu ta ra dáng một cảnh sát trưởng ghê vậy đó  X =  [None, 20, 47, 40, 38, 98, 40, 39, 87, 10, 135, 6, 40, 83, 0, 96, 104, 40, 53, 132, 12, 40, 96, 39, 68, 87, 40, 36, 33, 98, 40, 55, 37, 64, 67, 40, 39, 84, 127, 40, 12, 33, 40, 92, 33, 40,


Iteration: 1201, Loss: 2.782294

 chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chún.
 như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như .
Thiền như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng.
 như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như .
 như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng như và chúng n


Iteration: 2401, Loss: 2.179434

 thương cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũ.
 cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng cũng.
 chúc nhiên bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay .
 cảm bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn bay bàn.
O thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên thiên 


Iteration: 3601, Loss: 2.177989

Hục chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện.
Hà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho ngh.
Cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe c.
 thơ thơ nhà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện .
 cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho nghe chuyện bà cho n

KeyboardInterrupt: 

In [ ]:
from torch import nn
rnn = nn.RNN(vocab_size, 50, max_str_len, batch_first=True)
parameters = list(rnn.parameters())
len(parameters)